In [1]:
#import libraries

import numpy as np
import pandas as pd
from sklearn.neighbors import DistanceMetric
from math import radians
from geopy import distance
import sys
from math import dist

In [2]:
#add accidents dataset into a dataframe
df_buildings = pd.read_csv('building_inventory_enrich.csv')

df_buildings

C:\Users\penpo\AppData\Local\Temp\ipykernel_22152\2418044373.py:2: DtypeWarning: Columns (1,40,44,47,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df_buildings = pd.read_csv('building_inventory_enrich.csv')


,id,pid_long,cm_id,building_typology,building_subtypology,use_class,sqft_class,year_built_class,st_num,st_name,...,insulate_exposed_ducts,insulate_exposed_pipes,interior_wall_insulation_blow_in,exterior_wall_insulation_at_replacement,exterior_wall_insulation,interior_wall_insulation_board,insulate_spandrel,asbestos,seal_elevator_vent_shafts,env_recommendation_count
0,88905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,f,f,f,f,f,f,f,f,f,0
1,51137,1600083000,NaN,Single-Family,"Single-Family, 1915-1949",Residential,"< 25,000 sqft",1915-1949,5,EVERETT,...,f,t,f,f,t,t,f,f,f,4
2,18045,202560000,NaN,Single-Family,"Single-Family, Pre-1915",Residential,"< 25,000 sqft",Pre-1915,43,ESSEX,...,f,t,f,f,f,t,f,f,f,3
3,94001,2200577000,NaN,Convention/Assembly,"Convention/Assembly, 1950-1979",Public & Quasi-Public,"< 25,000 sqft",1950-1979,525,WESTERN,...,f,f,f,f,f,f,f,f,f,0
4,92940,1600058000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,f,f,f,f,f,f,f,f,f,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98925,66236,1001363000,1.001363e+09,Multi-Family (2 units),"Multi-Family (2 units), 1980-present",Residential,"< 25,000 sqft",1980-present,NaN,NaN,...,f,f,f,f,f,f,f,f,f,0
98926,68980,2007422000,NaN,Single-Family,"Single-Family, 1980-present",Residential,"< 25,000 sqft",1980-present,19,AVALON,...,f,f,f,f,f,f,f,f,f,0
98927,95554,2203167050,NaN,Single-Family,"Single-Family, 1980-present",Residential,"< 25,000 sqft",1980-present,46,GERRISH,...,f,f,f,f,f,f,f,f,f,0
98928,58753,200894010,2.008940e+08,Multi-Family (2 units),"Multi-Family (2 units), 1980-present",Residential,"< 25,000 sqft",1980-present,NaN,NaN,...,f,f,f,f,f,f,f,f,f,0


In [3]:
df_buildings.columns

Index(['id', 'pid_long', 'cm_id', 'building_typology', 'building_subtypology',
       'use_class', 'sqft_class', 'year_built_class', 'st_num', 'st_name',
       ...
       'insulate_exposed_ducts', 'insulate_exposed_pipes',
       'interior_wall_insulation_blow_in',
       'exterior_wall_insulation_at_replacement', 'exterior_wall_insulation',
       'interior_wall_insulation_board', 'insulate_spandrel', 'asbestos',
       'seal_elevator_vent_shafts', 'env_recommendation_count'],
      dtype='object', length=107)

In [4]:
df_hotels = df_buildings[df_buildings['building_typology'] == 'Hotel']

df_hotels

,id,pid_long,cm_id,building_typology,building_subtypology,use_class,sqft_class,year_built_class,st_num,st_name,...,insulate_exposed_ducts,insulate_exposed_pipes,interior_wall_insulation_blow_in,exterior_wall_insulation_at_replacement,exterior_wall_insulation,interior_wall_insulation_board,insulate_spandrel,asbestos,seal_elevator_vent_shafts,env_recommendation_count
199,43735,602680100,NaN,Hotel,"Hotel, 1980-present",Commercial,">= 100,000 sqft",1980-present,606,CONGRESS,...,f,f,f,f,f,f,f,f,t,1
205,97181,303883000,NaN,Hotel,"Hotel, 1980-present",Commercial,"50,000-99,999 sqft",1980-present,56 - 68,DEVONSHIRE,...,f,f,f,f,f,f,f,f,t,1
218,95854,602674165,NaN,Hotel,"Hotel, 1980-present",Commercial,">= 100,000 sqft",1980-present,660,SUMMER,...,f,f,f,f,f,f,f,f,t,1
231,95368,305648000,NaN,Hotel,"Hotel, 1980-present",Commercial,"50,000-99,999 sqft",1980-present,240 - 242,TREMONT,...,f,f,f,f,f,f,f,f,t,1
241,94957,100549001,NaN,Hotel,"Hotel, 1950-1979",Commercial,">= 100,000 sqft",1950-1979,225,WM F MCCLELLAN,...,f,f,f,f,f,f,f,f,t,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96979,64647,303295000,NaN,Hotel,"Hotel, 1915-1949",Commercial,"< 25,000 sqft",1915-1949,247 243,HANOVER,...,f,f,f,f,f,f,f,f,f,0
97706,46935,401328000,NaN,Hotel,NaN,Commercial,"< 25,000 sqft",NaN,30 24,BELVIDERE,...,f,f,f,f,f,f,f,f,f,0
98001,43592,401117000,NaN,Hotel,"Hotel, Pre-1915",Commercial,">= 100,000 sqft",Pre-1915,350,STUART,...,f,f,f,f,f,f,f,f,f,0
98016,38416,500087000,NaN,Hotel,"Hotel, Pre-1915",Commercial,"< 25,000 sqft",Pre-1915,76 82,CHARLES STREET SOUTH,...,f,f,f,f,f,f,f,f,f,0


In [5]:
df_buildings.shape

(98930, 107)

In [6]:
#Create a subset:
df_hotels = df_hotels[['id', 'building_typology', 'st_num', 'st_name', 'zipcode', 'sqft', 'owner_list']]

df_hotels

,id,building_typology,st_num,st_name,zipcode,sqft,owner_list
199,43735,Hotel,606,CONGRESS,2210.0,538000.0,RBW OWNER LP
205,97181,Hotel,56 - 68,DEVONSHIRE,2109.0,85700.0,MHF DEVONSHIRE V LLC
218,95854,Hotel,660,SUMMER,2210.0,320000.0,PARCEL A DEVELOPMENT
231,95368,Hotel,240 - 242,TREMONT,2116.0,95000.0,TS LODGING INC
241,94957,Hotel,225,WM F MCCLELLAN,2128.0,115045.0,SLUMBER TIME LLC
...,...,...,...,...,...,...,...
96979,64647,Hotel,247 243,HANOVER,2113.0,7864.0,BRICCO REALTY LLC
97706,46935,Hotel,30 24,BELVIDERE,2115.0,1.0,PIM BOSTON BACK BAY LLC
98001,43592,Hotel,350,STUART,2116.0,189200.0,LBV HOTEL LLC
98016,38416,Hotel,76 82,CHARLES STREET SOUTH,2116.0,19602.0,78 CHARLES STREET PROPCO LLC


In [7]:
#remove zero values from lat and long

df_hotels= df_hotels[df_hotels['zipcode'] != 0]

df_hotels

,id,building_typology,st_num,st_name,zipcode,sqft,owner_list
199,43735,Hotel,606,CONGRESS,2210.0,538000.0,RBW OWNER LP
205,97181,Hotel,56 - 68,DEVONSHIRE,2109.0,85700.0,MHF DEVONSHIRE V LLC
218,95854,Hotel,660,SUMMER,2210.0,320000.0,PARCEL A DEVELOPMENT
231,95368,Hotel,240 - 242,TREMONT,2116.0,95000.0,TS LODGING INC
241,94957,Hotel,225,WM F MCCLELLAN,2128.0,115045.0,SLUMBER TIME LLC
...,...,...,...,...,...,...,...
96979,64647,Hotel,247 243,HANOVER,2113.0,7864.0,BRICCO REALTY LLC
97706,46935,Hotel,30 24,BELVIDERE,2115.0,1.0,PIM BOSTON BACK BAY LLC
98001,43592,Hotel,350,STUART,2116.0,189200.0,LBV HOTEL LLC
98016,38416,Hotel,76 82,CHARLES STREET SOUTH,2116.0,19602.0,78 CHARLES STREET PROPCO LLC


In [8]:
#remove nan values
df_hotels.dropna(axis='rows', how='any', inplace=True)

df_hotels.shape

(82, 7)

In [9]:
convert_dict = {'id':int, 'zipcode': int,'sqft': int }

df_hotels = df_hotels.astype(convert_dict)

df_hotels.dtypes


id                    int32
building_typology    object
st_num               object
st_name              object
zipcode               int32
sqft                  int32
owner_list           object
dtype: object

In [10]:
df_hotels['Address'] = df_hotels['st_num'].str.cat(df_hotels['st_name'],sep=", ")

df_hotels.head()

,id,building_typology,st_num,st_name,zipcode,sqft,owner_list,Address
199,43735,Hotel,606,CONGRESS,2210,538000,RBW OWNER LP,"606, CONGRESS"
205,97181,Hotel,56 - 68,DEVONSHIRE,2109,85700,MHF DEVONSHIRE V LLC,"56 - 68, DEVONSHIRE"
218,95854,Hotel,660,SUMMER,2210,320000,PARCEL A DEVELOPMENT,"660, SUMMER"
231,95368,Hotel,240 - 242,TREMONT,2116,95000,TS LODGING INC,"240 - 242, TREMONT"
241,94957,Hotel,225,WM F MCCLELLAN,2128,115045,SLUMBER TIME LLC,"225, WM F MCCLELLAN"


In [11]:
df_hotels['Address'] = df_hotels['Address'] + ', ' + 'Boston, Massachusetts'

df_hotels.head()

,id,building_typology,st_num,st_name,zipcode,sqft,owner_list,Address
199,43735,Hotel,606,CONGRESS,2210,538000,RBW OWNER LP,"606, CONGRESS, Boston, Massachusetts"
205,97181,Hotel,56 - 68,DEVONSHIRE,2109,85700,MHF DEVONSHIRE V LLC,"56 - 68, DEVONSHIRE, Boston, Massachusetts"
218,95854,Hotel,660,SUMMER,2210,320000,PARCEL A DEVELOPMENT,"660, SUMMER, Boston, Massachusetts"
231,95368,Hotel,240 - 242,TREMONT,2116,95000,TS LODGING INC,"240 - 242, TREMONT, Boston, Massachusetts"
241,94957,Hotel,225,WM F MCCLELLAN,2128,115045,SLUMBER TIME LLC,"225, WM F MCCLELLAN, Boston, Massachusetts"


In [12]:
#Extract the address as dictionaries

hotel_address = df_hotels[['id','Address']]
addresses_dict = hotel_address.set_index('id').T.to_dict('list')

addresses_dict

{43735: ['606, CONGRESS, Boston, Massachusetts'],
 97181: ['56 - 68, DEVONSHIRE, Boston, Massachusetts'],
 95854: ['660, SUMMER, Boston, Massachusetts'],
 95368: ['240 - 242, TREMONT, Boston, Massachusetts'],
 94957: ['225, WM F MCCLELLAN, Boston, Massachusetts'],
 94650: ['400, SOLDIERS FIELD, Boston, Massachusetts'],
 61672: ['6 8, W BROADWAY, Boston, Massachusetts'],
 79037: ['39, DALTON, Boston, Massachusetts'],
 96207: ['155, PORTLAND, Boston, Massachusetts'],
 885: ['90, TREMONT, Boston, Massachusetts'],
 53226: ['120, HUNTINGTON, Boston, Massachusetts'],
 89133: ['704 712, BOYLSTON, Boston, Massachusetts'],
 6231: ['15, ARLINGTON, Boston, Massachusetts'],
 18389: ['54 66, BATTERYMARCH, Boston, Massachusetts'],
 51992: ['5, BLOSSOM, Boston, Massachusetts'],
 77755: ['40, DALTON, Boston, Massachusetts'],
 24500: ['159 175, DEVONSHIRE, Boston, Massachusetts'],
 55413: ['271 279, TREMONT, Boston, Massachusetts'],
 18402: ['1 11, COURT, Boston, Massachusetts'],
 16257: ['60 74, TREMO

In [13]:
#extract lad and long for hotels

from geopy.geocoders import Nominatim

n = Nominatim(user_agent="df_hotels_1")
latitude_1 = []
longitude_1 = []
for id in df_hotels["id"]:
    address_value = addresses_dict[id]
    location = n.geocode(address_value[0])
    latitude_1.append(location.latitude if location else '')
    longitude_1.append(location.longitude if location else '')
df_hotels["hotel_lat"] = latitude_1
df_hotels["hotel_lng"] = longitude_1

In [14]:
df_hotels

,id,building_typology,st_num,st_name,zipcode,sqft,owner_list,Address,hotel_lat,hotel_lng
199,43735,Hotel,606,CONGRESS,2210,538000,RBW OWNER LP,"606, CONGRESS, Boston, Massachusetts",42.347952,-71.039623
205,97181,Hotel,56 - 68,DEVONSHIRE,2109,85700,MHF DEVONSHIRE V LLC,"56 - 68, DEVONSHIRE, Boston, Massachusetts",42.35622,-71.057356
218,95854,Hotel,660,SUMMER,2210,320000,PARCEL A DEVELOPMENT,"660, SUMMER, Boston, Massachusetts",42.345525,-71.037938
231,95368,Hotel,240 - 242,TREMONT,2116,95000,TS LODGING INC,"240 - 242, TREMONT, Boston, Massachusetts",42.334353,-71.089555
241,94957,Hotel,225,WM F MCCLELLAN,2128,115045,SLUMBER TIME LLC,"225, WM F MCCLELLAN, Boston, Massachusetts",42.387389,-71.016344
...,...,...,...,...,...,...,...,...,...,...
96979,64647,Hotel,247 243,HANOVER,2113,7864,BRICCO REALTY LLC,"247 243, HANOVER, Boston, Massachusetts",,
97706,46935,Hotel,30 24,BELVIDERE,2115,1,PIM BOSTON BACK BAY LLC,"30 24, BELVIDERE, Boston, Massachusetts",42.345785,-71.08267
98001,43592,Hotel,350,STUART,2116,189200,LBV HOTEL LLC,"350, STUART, Boston, Massachusetts",42.349282,-71.072837
98016,38416,Hotel,76 82,CHARLES STREET SOUTH,2116,19602,78 CHARLES STREET PROPCO LLC,"76 82, CHARLES STREET SOUTH, Boston, Massachus...",42.351394,-71.067049


In [15]:
#Extract the coordinates as dictionaries

Hotel_coordinates = df_hotels[['id','hotel_lat','hotel_lng','Address']]
hotel_dict = Hotel_coordinates.set_index('id').T.to_dict('list')

hotel_dict

{43735: [42.3479524, -71.0396228, '606, CONGRESS, Boston, Massachusetts'],
 97181: [42.3562203,
  -71.0573561,
  '56 - 68, DEVONSHIRE, Boston, Massachusetts'],
 95854: [42.3455247, -71.03793848361921, '660, SUMMER, Boston, Massachusetts'],
 95368: [42.3343528, -71.0895549, '240 - 242, TREMONT, Boston, Massachusetts'],
 94957: [42.3873885,
  -71.01634382443356,
  '225, WM F MCCLELLAN, Boston, Massachusetts'],
 94650: [42.3602992,
  -71.11815850541063,
  '400, SOLDIERS FIELD, Boston, Massachusetts'],
 61672: [42.3415431, -71.0550616, '6 8, W BROADWAY, Boston, Massachusetts'],
 79037: [42.34651075, -71.08386718567789, '39, DALTON, Boston, Massachusetts'],
 96207: [42.36420135,
  -71.06135479193117,
  '155, PORTLAND, Boston, Massachusetts'],
 885: [42.3574423, -71.0607367, '90, TREMONT, Boston, Massachusetts'],
 53226: [42.3457259, -71.0809307, '120, HUNTINGTON, Boston, Massachusetts'],
 89133: [42.3465199, -71.091336, '704 712, BOYLSTON, Boston, Massachusetts'],
 6231: [42.35282695,
  -71

In [16]:
#add dataset into a dataframe
df_snowparking = pd.read_csv('Snow_Emergency_Parking_Lots_Enrich.csv', encoding='cp1252')

df_snowparking

,X,Y,ObjectId,Name,Address,Spaces,Fee,Comments,Phone,Neighborhood,Maxspaces,Hours,Longitude,Latitude
0,-7918411.415,5213945.934,1,Municipal Lot #003,115 Harvard Ave,60,No Charge,NaN,NaN,NaN,0,NaN,-71.1323,42.35261
1,-7920693.465,5213501.578,2,Municipal Lot #002,398 Market St,42,No Charge,NaN,NaN,NaN,0,NaN,-71.1528,42.34966
2,-7917331.616,5215700.967,3,Harvard University/Soldiers Field Park Garage,111 Western Ave,0,No Charge,Only Allston and Brighton residents can park a...,617-496-6400,NaN,0,NaN,-71.1226,42.36426
3,-7921071.951,5203873.964,4,Municipal Lot #034,1891 Centre Street,18,No Charge,NaN,NaN,NaN,0,NaN,-71.1562,42.28571
4,-7920838.180,5203887.506,5,Municipal Lot #010,39-41 Corey St,53,No Charge,NaN,NaN,NaN,0,NaN,-71.1541,42.28580
5,-7910585.655,5214060.415,6,Lafayette Garage,1 Avenue De Lafayette,0,$10 for each 24-hour period,"Discounted parking is only for Chinatown, Down...",617-357-1987,NaN,0,NaN,-71.0620,42.35337
6,-7908860.203,5211509.010,7,Municipal Lot #018,450 West Broadway,60,No Charge,NaN,NaN,NaN,0,NaN,-71.0465,42.33643
7,-7907958.515,5211427.690,8,Municipal Lot #021,650-652 East Broadway,22,No Charge,NaN,NaN,NaN,0,NaN,-71.0384,42.33589
8,-7907680.216,5212805.713,9,BPDA/EDIC Garage,12 Drydock Ave,0,$1 for each night,Discounted rate for residents of South Boston ...,617-482-2487,NaN,0,NaN,-71.0359,42.34504
9,-7913402.038,5211611.415,10,Northeastern University/Renaissance Park Garage,835 Columbus Avenue,0,50% of the variable rate. The most you will pa...,"Discounted parking is for Roxbury, South End, ...",617-267-9851 X106,NaN,0,NaN,-71.0873,42.33711


In [17]:
#create a subset of dataset 

df_snowsub = df_snowparking[['ObjectId','Name', 'Address', 'Spaces', 'Longitude', 'Latitude']]

df_snowsub

,ObjectId,Name,Address,Spaces,Longitude,Latitude
0,1,Municipal Lot #003,115 Harvard Ave,60,-71.1323,42.35261
1,2,Municipal Lot #002,398 Market St,42,-71.1528,42.34966
2,3,Harvard University/Soldiers Field Park Garage,111 Western Ave,0,-71.1226,42.36426
3,4,Municipal Lot #034,1891 Centre Street,18,-71.1562,42.28571
4,5,Municipal Lot #010,39-41 Corey St,53,-71.1541,42.28580
5,6,Lafayette Garage,1 Avenue De Lafayette,0,-71.0620,42.35337
6,7,Municipal Lot #018,450 West Broadway,60,-71.0465,42.33643
7,8,Municipal Lot #021,650-652 East Broadway,22,-71.0384,42.33589
8,9,BPDA/EDIC Garage,12 Drydock Ave,0,-71.0359,42.34504
9,10,Northeastern University/Renaissance Park Garage,835 Columbus Avenue,0,-71.0873,42.33711


In [18]:
convert_dict = {'ObjectId':int, 'Longitude': float ,'Latitude': float }

df_snowsub = df_snowsub.astype(convert_dict)

df_snowsub.dtypes

ObjectId       int32
Name          object
Address       object
Spaces         int64
Longitude    float64
Latitude     float64
dtype: object

In [19]:
#Extract the coordinates as dictionaries

snowparking_coordinates = df_snowsub[['ObjectId','Longitude', 'Latitude']]
snowparking_dict = snowparking_coordinates.set_index('ObjectId').T.to_dict('list')

snowparking_dict

{1: [-71.1323, 42.35261],
 2: [-71.1528, 42.34966],
 3: [-71.1226, 42.36426],
 4: [-71.1562, 42.28571],
 5: [-71.1541, 42.2858],
 6: [-71.062, 42.35337],
 7: [-71.0465, 42.33643],
 8: [-71.0384, 42.33589],
 9: [-71.0359, 42.34504],
 10: [-71.0873, 42.33711],
 11: [-71.0857, 42.34041],
 12: [-71.0858, 42.3471],
 13: [-71.0509, 42.37664],
 14: [-71.072, 42.29138],
 15: [-71.0653, 42.31808],
 16: [-71.0606, 42.3024],
 17: [-71.0671, 42.31814],
 18: [-71.0661, 42.31817],
 19: [-71.0837, 42.30921],
 20: [-71.067, 42.30678],
 21: [-71.0717, 42.28958],
 22: [-71.0757, 42.32453],
 23: [-71.0372, 42.3744],
 24: [-71.0376, 42.37487],
 25: [-71.0377, 42.37558],
 26: [-71.1251, 42.25519],
 27: [-71.1235, 42.25654],
 28: [-71.0973, 42.31655],
 29: [-71.1151, 42.31108],
 30: [-71.1073, 42.32225],
 31: [-71.0988, 42.31565],
 32: [-71.1124, 42.31808],
 33: [-71.0927, 42.26873],
 34: [-71.0954, 42.26876],
 35: [-71.1287, 42.2875],
 36: [-71.0837, 42.33142],
 37: [-71.0831, 42.32922],
 38: [-71.0762, 42

In [20]:

import math

min_distance=[]
parking_name=[]
nearest_address=[]
spaces_available=[]
for addr_h,lat_h,lng_h in zip(df_hotels["Address"],df_hotels["hotel_lat"],df_hotels["hotel_lng"]): 
    dist = []
    for addr_p,lat_p,lng_p  in zip(df_snowsub["Address"],df_snowsub["Latitude"],df_snowsub["Longitude"]): 
        if lat_h == '' or lng_h == '' or lat_p == '' or lng_p == '':
            distance = 0
        else:
            distance = math.dist([float(lat_h), float(lng_h)], [float(lat_p), float(lng_p)])
        dist.append(distance)
    min_dist = min(dist)
    min_index = dist.index(min_dist)
    min_distance.append(min_dist)
    address=df_snowsub["Address"].iloc[min_index]
    nearest_address.append(str(address))
    parking_nm=df_snowsub["Name"].iloc[min_index]
    parking_name.append(str(parking_nm))
    space=df_snowsub["Spaces"].iloc[min_index]
    spaces_available.append(space)


In [21]:

#calculate    
df_hotels["minimum_distance"]= min_distance
df_hotels["Closest Snow Parking Address"]= nearest_address
df_hotels["Snor Parking Name"]= parking_name
df_hotels["Spaces"]= spaces_available

df_hotels

,id,building_typology,st_num,st_name,zipcode,sqft,owner_list,Address,hotel_lat,hotel_lng,minimum_distance,Closest Snow Parking Address,Snor Parking Name,Spaces
199,43735,Hotel,606,CONGRESS,2210,538000,RBW OWNER LP,"606, CONGRESS, Boston, Massachusetts",42.347952,-71.039623,0.004727,12 Drydock Ave,BPDA/EDIC Garage,0
205,97181,Hotel,56 - 68,DEVONSHIRE,2109,85700,MHF DEVONSHIRE V LLC,"56 - 68, DEVONSHIRE, Boston, Massachusetts",42.35622,-71.057356,0.005449,1 Avenue De Lafayette,Lafayette Garage,0
218,95854,Hotel,660,SUMMER,2210,320000,PARCEL A DEVELOPMENT,"660, SUMMER, Boston, Massachusetts",42.345525,-71.037938,0.002095,12 Drydock Ave,BPDA/EDIC Garage,0
231,95368,Hotel,240 - 242,TREMONT,2116,95000,TS LODGING INC,"240 - 242, TREMONT, Boston, Massachusetts",42.334353,-71.089555,0.003562,835 Columbus Avenue,Northeastern University/Renaissance Park Garage,0
241,94957,Hotel,225,WM F MCCLELLAN,2128,115045,SLUMBER TIME LLC,"225, WM F MCCLELLAN, Boston, Massachusetts",42.387389,-71.016344,0.024403,36 Bennington St,Municipal Lot #026,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96979,64647,Hotel,247 243,HANOVER,2113,7864,BRICCO REALTY LLC,"247 243, HANOVER, Boston, Massachusetts",,,0.000000,115 Harvard Ave,Municipal Lot #003,60
97706,46935,Hotel,30 24,BELVIDERE,2115,1,PIM BOSTON BACK BAY LLC,"30 24, BELVIDERE, Boston, Massachusetts",42.345785,-71.08267,0.001234,150 Huntington Ave,Greenhouse Parking Garage/Ultimate,0
98001,43592,Hotel,350,STUART,2116,189200,LBV HOTEL LLC,"350, STUART, Boston, Massachusetts",42.349282,-71.072837,0.000970,400 Stuart St,LAZ Parking/The Clarendon Parking Garage,0
98016,38416,Hotel,76 82,CHARLES STREET SOUTH,2116,19602,78 CHARLES STREET PROPCO LLC,"76 82, CHARLES STREET SOUTH, Boston, Massachus...",42.351394,-71.067049,0.001018,201 Stuart Street,Motor Mart Garage,0
